In [2]:
!pip install econml


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 7.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 9.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 14.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 MB 8.2 MB/s eta 0:00:00:00:0100:01

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import yfinance as yf
import pandas as pd
from econml.dml import LinearDML
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [4]:
# Download historical stock data for multiple stocks
tickers = ["AAPL", "MSFT", "GOOGL"]  # Example: Apple, Microsoft, and Google
start_date = "2020-01-01"
end_date = "2023-01-01"

# Fetch the closing prices
data = yf.download(tickers, start=start_date, end=end_date)["Close"]

# Compute daily returns as percentage change of the closing prices
daily_returns = data.pct_change().dropna()

[*********************100%%**********************]  3 of 3 completed


In [5]:
daily_returns.shape


In [11]:
daily_returns.head()


Ticker,AAPL,GOOGL,MSFT
Date,,,
2020-01-03,-0.009722,-0.005231,-0.012452
2020-01-06,0.007968,0.026654,0.002585
2020-01-07,-0.004703,-0.001932,-0.009118
2020-01-08,0.016086,0.007118,0.015928
2020-01-09,0.021241,0.010498,0.012493


In [6]:
# Prepare data for CATE estimation
# Let's assume we want to estimate the effect of MSFT returns (treatment) on AAPL returns (outcome)
# and we consider GOOGL returns as an additional feature (control variable).
X = daily_returns[['GOOGL']]  # Control variable
T = daily_returns['MSFT']     # Treatment
Y = daily_returns['AAPL']     # Outcome

In [8]:
help(LinearDML)


Help on class LinearDML in module econml.dml.dml:

class LinearDML(econml._cate_estimator.StatsModelsCateEstimatorMixin, DML)
 |  LinearDML(*, model_y='auto', model_t='auto', featurizer=None, treatment_featurizer=None, fit_cate_intercept=True, linear_first_stages='deprecated', discrete_outcome=False, discrete_treatment=False, categories='auto', cv=2, mc_iters=None, mc_agg='mean', random_state=None, allow_missing=False, enable_federation=False, use_ray=False, ray_remote_func_options=None)
 |  
 |  The Double ML Estimator with a low-dimensional linear final stage implemented as a statsmodel regression.
 |  
 |  Parameters
 |  ----------
 |  model_y: estimator, default ``'auto'``
 |      Determines how to fit the outcome to the features.
 |  
 |      - If ``'auto'``, the model will be the best-fitting of a set of linear and forest models
 |  
 |      - Otherwise, see :ref:`model_selection` for the range of supported options;
 |        if a single model is specified it should be a classifi

In [9]:
# Instantiate the model
# We use GradientBoostingRegressor for the model_y and model_t to capture non-linear relationships
# LinearRegression is used for the final model to estimate the CATE
model = LinearDML(model_y=GradientBoostingRegressor(),
                  model_t=GradientBoostingRegressor(),
                #   model_final=LinearRegression(),
                  featurizer=None)

# Fit the model
model.fit(Y, T, X=X)

# Estimate the treatment effects
te_estimates = model.effect(X)

# Create a DataFrame for the estimated effects
te_df = pd.DataFrame(data={'Estimated Effect': te_estimates}, index=X.index)



In [10]:

print(te_df.head())


            Estimated Effect
Date                        
2020-01-03          0.663510
2020-01-06          0.762509
2020-01-07          0.673755
2020-01-08          0.701851
2020-01-09          0.712347
